In [1]:
import os
import csv
import pandas as pd

from batid.services.guess_bdg_new import Guesser, GeocodeNameHandler, GeocodeAddressHandler

efa_iub_path = "EFA_IUB.csv"
efa_surface_path = "EFA_SURFACE.csv"
efa_path = "EFA.csv"

geocoded_addresses_path = "geocoded_addresses.csv"

guess_path = "guesses.json"

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [14]:
# EFA IUB

df = pd.read_csv(efa_iub_path, sep=";")

print(df.columns)
print(df.head(10))
print(f"{len(df)} lignes")

print("Dénomination bâtiment")
print(df["Dénomination bâtiment"].value_counts())

Index(['ID EFA', 'Péfixe', 'Section', 'Dénomination bâtiment', 'Parcelle',
       'Lot'],
      dtype='object')
   ID EFA Péfixe Section Dénomination bâtiment Parcelle       Lot
0    1812    000      BY            ARGENTEUIL     0589      0000
1    1812    000      BY            ARGENTEUIL     0628      0000
2    1812    000      BY            ARGENTEUIL     0630      0000
3    1813    115      BS           HIGH SQUARE     0168      0000
4    1813    115      BS           HIGH SQUARE     0121      0000
5    1535    107      AD                230STG     0010  STEINWAY
6    1536    107      AD                230STG     0010     AARPI
7    1803    108      AQ                   9RP     0077        ID
8    1807    108      AQ                   9RP     0077      GIMD
9    1806    108      AQ                   7RP     0077       7RP
413482 lignes
Dénomination bâtiment
Dénomination bâtiment
1                             2204
CENTRE COMMERCIAL             1789
MAIRIE                        1476

In [15]:
# EFA Surface

df = pd.read_csv(efa_surface_path, sep=";")

print(df.columns)
print(df.head(10))
print(f"{len(df)} lignes")

Index(['ID Déclaration consommation', 'Année consommation', 'ID EFA',
       'Identifiant occupant', 'Nom occupant EFA',
       'Surface moyenne annuelle EFA (m²)'],
      dtype='object')
   ID Déclaration consommation Année consommation  ID EFA  \
0                        32622               2020   44726   
1                       256709               2020  148143   
2                       130290               2020  153793   
3                      1116939               2022  435727   
4                      1144326               2023   88752   
5                       872624               2022  130758   
6                      1276576               2023  228156   
7                       957092               2022  225056   
8                       795358               2022  225921   
9                       323652               2021  243745   

  Identifiant occupant                Nom occupant EFA  \
0       34181606401147                     CHAUSS'EXPO   
1       67850126300175  

In [25]:
# EFA

df = pd.read_csv(efa_path, sep=";")

print(df.columns)
print(df.head(10))
print(f"{len(df)} lignes")

# A propos de cas_assujetissement
print("Cas assujettissement")
print(df["Cas assujettissement"].value_counts())
print("Complément adresse OPR")
print(df["Complément adresse OPR"].value_counts())



/tmp/ipykernel_84/3040905977.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(efa_path, sep=";")


Index(['ID EFA', 'Cas assujettissement', 'Identifiant occupant',
       'Nom occupant EFA', 'Complément nom EFA', 'Dénomination EFA',
       'Adresse (Operat)', 'Complément adresse OPR', 'Commune OPR',
       'Code commune OPR', 'Code postal OPR', 'Siège SIRN',
       'Nom établissement SIRN', 'Enseigne établissement SIRN', 'Adresse SIRN',
       'Complt adresse SIRN', 'Code postal SIRN', 'Code commune SIRN',
       'Commune maj SIRN', 'Commune SIRN'],
      dtype='object')
   ID EFA Cas assujettissement        Identifiant occupant  \
0       5                    3              48458811600018   
1       6                   1A              77568873203099   
2       8                    2              80852873100025   
3       9                    2              80852873100025   
4      11                   1A              21200004600012   
5      28                   1A              83271794600012   
6      33                   1A              86680121000026   
7      34                

Il est probable qu'on utilise uniquement le fichier EFA.csv

Dedans, on a à dispo : 

- Nom occupant EFA
- Complément nom EFA

On pourra peut être les utiliser avec un Photon

- Les champs Adresse Operat à géocoder

Pour une recherche par adresse. 
On va devoir géocoder les adresses pour obtenir un lat/lng à utiliser avec Photon

# Géocoder les adresses

In [26]:
# On va tenter d'utiliser le géocodeur Photon mais on n'a pas de latitude/longitude.
# On va aller chercher lat/lng en géocodant les adresses fournies





df["full_address"] = df["Adresse (Operat)"].astype(str) + " " + df["Code postal OPR"].astype(str) + " " + df["Commune OPR"].astype(str)

new_df = df[['ID EFA', 'full_address' ]]
new_df.to_csv(geocoded_addresses_path)




In [29]:
# Split in chunks

chunks_size = 50_000
chunks = [new_df[i:i+chunks_size] for i in range(0, len(new_df), chunks_size)]

# Example: save each chunk into a separate CSV
for idx, chunk in enumerate(chunks):
    chunk.to_csv(f'output_chunk_{idx}.csv', index=False)

In [30]:
from batid.services.geocoders import BanBatchGeocoder
import glob
from io import StringIO

geocoder = BanBatchGeocoder()

files = sorted(glob.glob('output_chunk_*.csv'))
for file in files:
    print(file)
    response = geocoder.geocode_file(file, columns=["full_address"])
    response_csv = StringIO(response.text)
    df_geocoded = pd.read_csv(response_csv, sep=',')
    df_geocoded.to_csv(file, index=False)


    
        




output_chunk_0.csv
output_chunk_1.csv
output_chunk_2.csv
output_chunk_3.csv
output_chunk_4.csv
output_chunk_5.csv
output_chunk_6.csv


In [31]:
# merge all file into one
files = sorted(glob.glob('output_chunk_*.csv'))

df_list = [pd.read_csv(file) for file in files]

merged_df = pd.concat(df_list, ignore_index=True)

# Save into a new CSV
merged_df.to_csv(geocoded_addresses_path, index=False)

In [32]:
# remove the chunks
import os

for file in sorted(glob.glob('output_chunk_*.csv')):
    os.remove(file)

# Attacher les adresses géocodées aux données EFA

In [36]:
# On merge

efa_df = pd.read_csv(efa_path, sep=";")
address_df = pd.read_csv(geocoded_addresses_path)

merged_df = pd.merge(efa_df, address_df, on="ID EFA", how="inner")

/tmp/ipykernel_84/2424521250.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  efa_df = pd.read_csv(efa_path, sep=";")


   ID EFA Cas assujettissement Identifiant occupant          Nom occupant EFA  \
0       5                    3       48458811600018       ROUSSELOT ANGOULEME   
1       6                   1A       77568873203099       APF FRANCE HANDICAP   
2       8                    2       80852873100025  GROUPE ROCHER OPERATIONS   
3       9                    2       80852873100025  GROUPE ROCHER OPERATIONS   
4      11                   1A       21200004600012                    MAIRIE   

  Complément nom EFA Dénomination EFA              Adresse (Operat)  \
0                NaN              NaN  RUE SAINT MICHEL A ANGOULEME   
1                NaN              NaN  17 BOULEVARD AUGUSTE BLANQUI   
2                NaN              NaN          LA CROIX DES ARCHERS   
3                NaN              NaN          LA CROIX DES ARCHERS   
4     HOTEL DE VILLE              NaN                       PL FOCH   

  Complément adresse OPR Commune OPR Code commune OPR  ...  \
0                    NaN

In [38]:
print(merged_df.columns)
print(merged_df.head())
print(f"merged lines : {len(merged_df)}")

Index(['ID EFA', 'Cas assujettissement', 'Identifiant occupant',
       'Nom occupant EFA', 'Complément nom EFA', 'Dénomination EFA',
       'Adresse (Operat)', 'Complément adresse OPR', 'Commune OPR',
       'Code commune OPR', 'Code postal OPR', 'Siège SIRN',
       'Nom établissement SIRN', 'Enseigne établissement SIRN', 'Adresse SIRN',
       'Complt adresse SIRN', 'Code postal SIRN', 'Code commune SIRN',
       'Commune maj SIRN', 'Commune SIRN', 'full_address', 'longitude',
       'latitude', 'result_score', 'result_score_next', 'result_label',
       'result_type', 'result_id', 'result_housenumber', 'result_name',
       'result_street', 'result_postcode', 'result_city', 'result_context',
       'result_citycode', 'result_oldcitycode', 'result_oldcity',
       'result_district', 'result_status'],
      dtype='object')
   ID EFA Cas assujettissement Identifiant occupant          Nom occupant EFA  \
0       5                    3       48458811600018       ROUSSELOT ANGOULEME   
1

In [40]:
geocoded_efa_df = merged_df[merged_df["result_score"] >= 0.8]
print(f"geocoded lines : {len(geocoded_efa_df)}")

214065
geocoded lines : 214065


In [44]:
def row_to_input(row):
    return {
        "ext_id": row["ID EFA"],
        "lat": row["latitude"],
        "lng": row["longitude"],
        "name": row["Nom occupant EFA"],
        "ban_id": row["result_id"] if row["result_type"] == "housenumber" else None
    }

In [47]:
inputs = []

for _, row in geocoded_efa_df.iterrows():
    inputs.append(row_to_input(row))

guesser = Guesser()
guesser.create_work_file(inputs, guess_path)


In [4]:
guesser = Guesser(batch_size=500)
guesser.handlers = [
    GeocodeNameHandler(sleep_time=0, photon_url="http://host.docker.internal:2322/api/", bbox_apothem_in_meters=1000), 
    GeocodeAddressHandler()
]

guesser.guess_work_file(guess_path)

  0%|          | 0/429 [00:00<?, ?it/s]

OperationalError: SSL SYSCALL error: EOF detected
